In [1]:
import credentials as cr
from spotify_api import Spotify_api

%load_ext autoreload
%autoreload 2

## Application parameters

In [2]:
# redirect_uri = 'http://localhost/'
data_path = cr.DATA_PATH

sp = Spotify_api()

## Get datas using Spotify API  
I get my personnal spotify datas using spotify api and spotify_api.py class. The Pandas dataframes created are converted into csv files which will be used to create the sqlite database.

In [14]:
playlists_df = sp.get_playlists_with_tracks()
features_df = sp.get_playlist_features(playlists_df)

features_df.drop_duplicates()

tracks_df = sp.get_tracks(features_df)

playlists_df.to_csv(data_path + 'playlists.csv', encoding='utf-8')
features_df.to_csv(data_path + 'features.csv', encoding='utf-8')
tracks_df.to_csv(data_path + 'tracks.csv', encoding='utf-8')

## DataBase connect and creation